# Preprocess ODK data to organized tables

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import os
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import DATA_DIR, GCP_PROJ_ID
from src.biomass_inventory import (
    extract_trees,
    extract_stumps,
    extract_dead_trees_class1,
    extract_dead_trees_class2s,
    extract_dead_trees_class2t,
    extract_ldw_with_hollow,
    extract_ldw_wo_hollow,
)

In [3]:
# Variables
URL = "https://api.ona.io/api/v1/data/763932.csv"
FILE_RAW = DATA_DIR / "csv" / "biomass_inventory_raw.csv"
CARBON_POOLS_OUTDIR = DATA_DIR / "csv" / "carbon_pools"
NESTS = [2, 3, 4]

# BigQuery Variables
DATASET_ID = "biomass_inventory"
IF_EXISTS = "replace"

In [4]:
# Create output directory
CARBON_POOLS_OUTDIR.mkdir(parents=True, exist_ok=True)

# Get Data from ONA

In [5]:
column_types = {
    col: str
    for col in (
        28,
        399,
        400,
        407,
        408,
        415,
        416,
        845,
        846,
        853,
        854,
        861,
        862,
        869,
        870,
        877,
        878,
        885,
        886,
        893,
        894,
        901,
        902,
        909,
        910,
        1179,
        1180,
        1187,
        1188,
        1195,
        1196,
        1203,
        1204,
        1211,
        1212,
        1219,
        1220,
        1286,
        1337,
        1342,
        1347,
        1352,
        1357,
        1362,
        1378,
        1392,
    )
}

In [6]:
if FILE_RAW.exists():
    data = pd.read_csv(FILE_RAW, dtype=column_types)
else:
    urllib.request.urlretrieve(URL, FILE_RAW)
    data = pd.read_csv(FILE_RAW, dtype=column_types)

## Add a unique ID

In [7]:
# Create a new column with "1" for Primary and "2" for Backup
data["plot_type_short"] = data["plot_info/plot_type"].apply(
    lambda x: "1" if x == "Primary" else "2"
)

# Extract subplot letters (assuming they are included in the 'plot_info.sub_plot' column)
data["subplot_letter"] = data["plot_info/sub_plot"].str.replace("sub_plot", "")

# Create the unique ID by concatenating the specified columns
data["unique_id"] = (
    data["plot_info/plot_code_nmbr"].astype(str)
    + data["subplot_letter"]
    + data["plot_type_short"]
)

# Extract Plot info

In [8]:
plot_info_cols = [
    "unique_id",
    "plot_info/data_recorder",
    "plot_info/team_no",
    "plot_info/plot_code_nmbr",
    "plot_info/plot_type",
    "plot_info/sub_plot",
    "plot_info/yes_no",
    "plot_shift/sub_plot_shift",
    "plot_GPS/GPS_waypt",
    "plot_GPS/GPS_id",
    "plot_GPS/GPS",
    "plot_GPS/_GPS_latitude",
    "plot_GPS/_GPS_longitude",
    "plot_GPS/_GPS_altitude",
    "plot_GPS/_GPS_precision",
    "plot_GPS/photo",
    "access/access_reason/slope",
    "access/access_reason/danger",
    "access/access_reason/distance",
    "access/access_reason/water",
    "access/access_reason/prohibited",
    "access/access_reason/other",
    "access/manual_reason",
    "lc_data/lc_type",
    "lc_class/lc_class",
    "lc_class/lc_class_other",
    "disturbance/disturbance_yesno",
    "disturbance_data/disturbance_type",
    "disturbance_class/disturbance_class",
    "slope/slope",
    "canopy/avg_height",
    "canopy/can_cov",
]

In [9]:
plot_info = data[plot_info_cols].copy()

In [10]:
# rename columns
plot_info_cols = {
    "plot_info/data_recorder": "data_recorder",
    "plot_info/team_no": "team_no",
    "plot_info/plot_code_nmbr": "plot_code_nmbr",
    "plot_info/plot_type": "plot_type",
    "plot_info/sub_plot": "sub_plot",
    "plot_info/yes_no": "yes_no",
    "plot_shift/sub_plot_shift": "sub_plot_shift",
    "plot_GPS/GPS_waypt": "GPS_waypt",
    "plot_GPS/GPS_id": "GPS_id",
    "plot_GPS/GPS": "GPS",
    "plot_GPS/_GPS_latitude": "GPS_latitude",
    "plot_GPS/_GPS_longitude": "GPS_longitude",
    "plot_GPS/_GPS_altitude": "GPS_altitude",
    "plot_GPS/_GPS_precision": "GPS_precision",
    "plot_GPS/photo": "photo",
    "access/access_reason/slope": "access_reason_slope",
    "access/access_reason/danger": "access_reason_danger",
    "access/access_reason/distance": "access_reason_distance",
    "access/access_reason/water": "access_reason_water",
    "access/access_reason/prohibited": "access_reason_prohibited",
    "access/access_reason/other": "access_reason_other",
    "access/manual_reason": "manual_reason",
    "lc_data/lc_type": "lc_type",
    "lc_class/lc_class": "lc_class",
    "lc_class/lc_class_other": "lc_class_other",
    "disturbance/disturbance_yesno": "disturbance_yesno",
    "disturbance_data/disturbance_type": "disturbance_type",
    "disturbance_class/disturbance_class": "disturbance_class",
    "slope/slope": "slope",
    "canopy/avg_height": "canopy_avg_height",
    "canopy/can_cov": "canopy_cover",
}

In [11]:
plot_info.rename(columns=plot_info_cols, inplace=True)

### Set correct data types

In [12]:
column_types = {
    "unique_id": str,
    "data_recorder": str,
    "team_no": int,
    "plot_code_nmbr": int,
    "plot_type": str,
    "sub_plot": str,
    "yes_no": str,
    "sub_plot_shift": str,
    "GPS": str,
    "photo": str,
    "access_reason_slope": str,
    "access_reason_danger": str,
    "access_reason_distance": str,
    "access_reason_water": str,
    "access_reason_prohibited": str,
    "access_reason_other": str,
    "manual_reason": str,
    "lc_type": str,
    "lc_class": str,
    "lc_class_other": str,
    "disturbance_yesno": str,
    "disturbance_type": str,
    "disturbance_class": str,
}

In [13]:
plot_info = plot_info.astype(column_types)

### Compress access reasons to one column

In [14]:
plot_info["access_reason"] = np.nan
plot_info["access_reason"] = plot_info["access_reason"].astype(str)
for index, row in plot_info.iterrows():
    if row["access_reason_slope"] == "True":
        plot_info.loc[index, "access_reason"] = "slope"
    elif row["access_reason_danger"] == "True":
        plot_info.loc[index, "access_reason"] = "danger"
    elif row["access_reason_distance"] == "True":
        plot_info.loc[index, "access_reason"] = "distance"
    elif row["access_reason_water"] == "True":
        plot_info.loc[index, "access_reason"] = "water"
    elif row["access_reason_prohibited"] == "True":
        plot_info.loc[index, "access_reason"] = "prohibited"
    elif row["access_reason_other"] == "True":
        plot_info.loc[index, "access_reason"] = row["manual_reason"]

In [15]:
# Categorize manual reasons
plot_info.loc[plot_info.access_reason == "90 degree slope ", "access_reason"] = "slope"
plot_info.loc[
    plot_info.access_reason == "Slippery due to rainfall and sharp stones..too risky.",
    "access_reason",
] = "danger"
plot_info.loc[
    plot_info.access_reason == "Creek plot and slope 90 degree", "access_reason"
] = "slope"
plot_info.loc[
    plot_info.access_reason == "Near creek 90 degree slope", "access_reason"
] = "slope"

In [16]:
# drop access_reason columns
plot_info.drop(
    columns=[
        "access_reason_slope",
        "access_reason_danger",
        "access_reason_distance",
        "access_reason_water",
        "access_reason_prohibited",
        "access_reason_other",
    ],
    inplace=True,
)

In [17]:
plot_info.access_reason.value_counts()

access_reason
nan       634
slope      31
danger      9
Name: count, dtype: int64

In [18]:
plot_info.info(), plot_info.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   unique_id          674 non-null    object 
 1   data_recorder      674 non-null    object 
 2   team_no            674 non-null    int64  
 3   plot_code_nmbr     674 non-null    int64  
 4   plot_type          674 non-null    object 
 5   sub_plot           674 non-null    object 
 6   yes_no             674 non-null    object 
 7   sub_plot_shift     674 non-null    object 
 8   GPS_waypt          634 non-null    float64
 9   GPS_id             634 non-null    float64
 10  GPS                674 non-null    object 
 11  GPS_latitude       577 non-null    float64
 12  GPS_longitude      577 non-null    float64
 13  GPS_altitude       577 non-null    float64
 14  GPS_precision      577 non-null    float64
 15  photo              674 non-null    object 
 16  manual_reason      674 non

(None,
   unique_id data_recorder  team_no  plot_code_nmbr plot_type   sub_plot  \
 0     308D2         Steve        1             308   primary  sub_plotD   
 1     308A2         Steve        1             308   primary  sub_plotA   
 
   yes_no sub_plot_shift  GPS_waypt  GPS_id  ...  lc_type  lc_class  \
 0    yes       no_shift        7.0     1.0  ...  f_plant        FC   
 1    yes       no_shift        8.0     1.0  ...    f_nat       MDF   
 
    lc_class_other  disturbance_yesno  disturbance_type disturbance_class  \
 0             nan                nan               nan               nan   
 1             nan                 no               nan               nan   
 
   slope canopy_avg_height canopy_cover access_reason  
 0  23.0              12.0          3.0           nan  
 1  13.0               8.0          4.0           nan  
 
 [2 rows x 27 columns])

## Export data and upload to BQ

In [19]:
# Export CSV
if len(plot_info) != 0:
    plot_info.to_csv(CARBON_POOLS_OUTDIR / "plot_info.csv", index=False)

In [20]:
# Upload to BQ
if len(plot_info) != 0:
    pandas_gbq.to_gbq(
        plot_info,
        f"{DATASET_ID}.plot_info",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 9510.89it/s]


# Extract info per carbon pool

# Saplings, Non tree vegetation and litter

In [21]:
cols = plot_info_cols = [
    "unique_id",
    "sapling_data/count_saplings",
    "ntv_data/litter_data/litter_bag_weight",
    "ntv_data/litter_data/litter_sample_weight",
    "ntv_data/ntv_bag_weight",
    "ntv_data/ntv_sample_weight",
]

In [22]:
# rename columns
col_names = {
    "sapling_data/count_saplings": "count_saplings",
    "ntv_data/litter_data/litter_bag_weight": "litter_bag_weight",
    "ntv_data/litter_data/litter_sample_weight": "litter_sample_weight",
    "ntv_data/ntv_bag_weight": "ntv_bag_weight",
    "ntv_data/ntv_sample_weight": "ntv_sample_weight",
}

In [23]:
ntv = data[cols].copy()

In [24]:
ntv.rename(columns=col_names, inplace=True)

In [25]:
ntv.info(), ntv.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_id             674 non-null    object 
 1   count_saplings        589 non-null    float64
 2   litter_bag_weight     620 non-null    float64
 3   litter_sample_weight  620 non-null    float64
 4   ntv_bag_weight        620 non-null    float64
 5   ntv_sample_weight     620 non-null    float64
dtypes: float64(5), object(1)
memory usage: 31.7+ KB


(None,
   unique_id  count_saplings  litter_bag_weight  litter_sample_weight  \
 0     308D2             2.0               70.0                 770.0   
 1     308A2             NaN               50.0                 260.0   
 
    ntv_bag_weight  ntv_sample_weight  
 0            70.0              870.0  
 1            50.0              110.0  )

## Export data and upload to BQ

In [26]:
# Export CSV
if len(ntv) != 0:
    ntv.to_csv(CARBON_POOLS_OUTDIR / "saplings_ntv_litter.csv", index=False)

In [27]:
# Upload to BQ
if len(ntv) != 0:
    pandas_gbq.to_gbq(
        ntv,
        f"{DATASET_ID}.saplings_ntv_litter",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 9892.23it/s]


# Living Trees

In [28]:
trees = extract_trees(data, NESTS)

In [29]:
trees.info(), trees.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6579 entries, 0 to 6578
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     6579 non-null   object 
 1   nest          6579 non-null   int64  
 2   species_name  5718 non-null   float64
 3   family_name   1330 non-null   float64
 4   DBH           6579 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 257.1+ KB


(None,
   unique_id  nest  species_name  family_name   DBH
 0     308D2     2           NaN         25.0  10.8
 1     308D2     2           NaN         25.0  17.3)

In [30]:
trees.describe()

,nest,species_name,family_name,DBH
count,6579.000000,5718.000000,1330.000000,6579.000000
mean,2.736434,326.618573,84.354135,40.478231
std,0.713525,269.467916,228.708801,25.398493
min,2.000000,2.000000,2.000000,10.000000
25%,2.000000,194.000000,22.000000,19.800000
50%,3.000000,280.000000,22.000000,35.900000
75%,3.000000,313.000000,33.000000,52.900000
max,4.000000,999.000000,999.000000,199.000000


## Export data and upload to BQ

In [31]:
# Export to CSV
trees.to_csv(CARBON_POOLS_OUTDIR / "trees.csv", index=False)

In [32]:
# Upload to BQ
pandas_gbq.to_gbq(
    trees, f"{DATASET_ID}.trees", project_id=GCP_PROJ_ID, if_exists=IF_EXISTS
)

100%|██████████| 1/1 [00:00<00:00, 10754.63it/s]


# Tree Stumps

[delete when fixed] Note: removed `'biomass_per_kg_tree': [biomass_per_kg_tree],`. In the original code there was a placeholder column created, this can be added later in the process when biomass per tree is actually calculated

In [33]:
stumps = extract_stumps(data, NESTS)

In [34]:
stumps.info(), stumps.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unique_id      1754 non-null   object 
 1   nest           1754 non-null   int64  
 2   Diam1          1754 non-null   float64
 3   Diam2          1754 non-null   float64
 4   slope          1754 non-null   float64
 5   height         1754 non-null   float64
 6   cut_cl         1754 non-null   object 
 7   hollow_go      1754 non-null   object 
 8   hollow_d1      171 non-null    float64
 9   hollow_d2      171 non-null    float64
 10  stump_density  1754 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 150.9+ KB


(None,
   unique_id  nest  Diam1  Diam2  slope  height   cut_cl hollow_go  hollow_d1  \
 0     308C2     2   30.0   29.0   43.0    18.0  saw_axe        no        NaN   
 1     249B2     2   15.0   10.0   51.0    80.0  saw_axe        no        NaN   
 
    hollow_d2  stump_density  
 0        NaN            1.0  
 1        NaN            3.0  )

In [35]:
stumps.describe()

,nest,Diam1,Diam2,slope,height,hollow_d1,hollow_d2,stump_density
count,1754.000000,1754.000000,1754.000000,1754.000000,1754.000000,171.000000,171.000000,1754.000000
mean,3.111745,40.575610,36.617423,32.697834,83.215314,36.059532,33.773450,2.128848
std,0.636456,28.664704,24.662826,18.110894,40.670748,23.908597,21.810063,0.791743
min,2.000000,10.000000,10.000000,1.000000,1.500000,5.000000,5.000000,1.000000
25%,3.000000,20.000000,18.700000,16.000000,52.000000,18.550000,18.250000,1.000000
50%,3.000000,32.000000,30.000000,33.000000,80.000000,30.000000,30.000000,2.000000
75%,4.000000,53.725000,48.875000,47.000000,110.000000,48.000000,43.000000,3.000000
max,4.000000,195.000000,198.000000,80.000000,199.100000,160.000000,150.000000,3.000000


## Export data and upload to BQ

In [36]:
# Export to CSV
stumps.to_csv(CARBON_POOLS_OUTDIR / "stumps.csv", index=False)

In [37]:
# Upload to BQ
pandas_gbq.to_gbq(
    stumps, f"{DATASET_ID}.stumps", project_id=GCP_PROJ_ID, if_exists=IF_EXISTS
)

100%|██████████| 1/1 [00:00<00:00, 9000.65it/s]


# Dead Trees: Class 1

In [38]:
dead_trees_c1 = extract_dead_trees_class1(data, NESTS)

No class 1 dead trees found in nest 2


In [39]:
dead_trees_c1.info(), dead_trees_c1.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     2 non-null      object 
 1   nest          2 non-null      int64  
 2   species_name  2 non-null      float64
 3   DBH_cl1       2 non-null      float64
 4   class         2 non-null      int64  
 5   subclass      2 non-null      object 
dtypes: float64(2), int64(2), object(2)
memory usage: 228.0+ bytes


(None,
   unique_id  nest  species_name  DBH_cl1  class subclass
 0     290C2     3         145.0     38.0      1      n/a
 1     290C2     4         177.0     58.8      1      n/a)

In [40]:
dead_trees_c1.describe()

,nest,species_name,DBH_cl1,class
count,2.000000,2.000000,2.000000,2.0
mean,3.500000,161.000000,48.400000,1.0
std,0.707107,22.627417,14.707821,0.0
min,3.000000,145.000000,38.000000,1.0
25%,3.250000,153.000000,43.200000,1.0
50%,3.500000,161.000000,48.400000,1.0
75%,3.750000,169.000000,53.600000,1.0
max,4.000000,177.000000,58.800000,1.0


## Export data and upload to BQ

In [41]:
dead_trees_c1.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class1.csv", index=False)

In [42]:
# Upload to BQ
pandas_gbq.to_gbq(
    dead_trees_c1,
    f"{DATASET_ID}.dead_trees_c1",
    project_id=GCP_PROJ_ID,
    if_exists=IF_EXISTS,
)

100%|██████████| 1/1 [00:00<00:00, 17848.10it/s]


# Dead Trees: Class 2 - short

In [43]:
dead_trees_c2s = extract_dead_trees_class2s(data, NESTS)

No dead trees of class 2 found in nest 2
No dead trees of class 2 found in nest 3
No dead trees of class 2 found in nest 4


In [44]:
dead_trees_c2s.info(), dead_trees_c2s.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


(None,
 Empty DataFrame
 Columns: []
 Index: [])

## Export data and upload to BQ

In [45]:
# Export CSV
if len(dead_trees_c2s) != 0:
    dead_trees_c2s.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class2.csv", index=False)

In [46]:
# Upload to BQ
if len(dead_trees_c2s) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c2s,
        f"{DATASET_ID}.dead_trees_c2s",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

# Dead Trees: Class 2 - Tall

In [47]:
dead_trees_c2t = extract_dead_trees_class2t(data, NESTS)

In [48]:
dead_trees_c2t.info(), dead_trees_c2t.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     254 non-null    object 
 1   nest          254 non-null    int64  
 2   species_name  201 non-null    float64
 3   family_name   12 non-null     float64
 4   dbh_tall      254 non-null    float64
 5   db_tall       254 non-null    float64
 6   tall_density  254 non-null    float64
 7   slope_t_tall  254 non-null    float64
 8   slope_b_tall  254 non-null    float64
 9   dist_t_tall   254 non-null    float64
 10  class         254 non-null    int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 22.0+ KB


(None,
   unique_id  nest  species_name  family_name  dbh_tall  db_tall  tall_density  \
 0     368A2     2         999.0          NaN      21.9     24.1           1.0   
 1     281A2     2           NaN          NaN      32.6     40.8           1.0   
 
    slope_t_tall  slope_b_tall  dist_t_tall  class  
 0          61.0          56.0         10.0      2  
 1         115.0          20.0          9.3      2  )

## Export data and upload to BQ

In [49]:
# Export CSV
if len(dead_trees_c2t) != 0:
    dead_trees_c2t.to_csv(
        CARBON_POOLS_OUTDIR / "dead_trees_class2_tall.csv", index=False
    )

In [50]:
# Upload to BQ
if len(dead_trees_c2t) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c2t,
        f"{DATASET_ID}.dead_trees_c2t",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 12052.60it/s]


# Lying Deadwood: Hollow

In [51]:
ldw_hollow = extract_ldw_with_hollow(data)

In [52]:
ldw_hollow.info(), ldw_hollow.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   15 non-null     object 
 1   repetition  15 non-null     int64  
 2   type        15 non-null     object 
 3   class       15 non-null     object 
 4   hollow_d1   15 non-null     float64
 5   hollow_d2   15 non-null     float64
 6   diameter    15 non-null     float64
 7   density     15 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ KB


(None,
   unique_id  repetition type class  hollow_d1  hollow_d2  diameter  density
 0     249D2           1  tr2   MDF       12.0       10.0      16.7      3.0
 1     290A2           2  tr2   MCB       68.0       28.0      62.0      1.0)

## Export data and upload to BQ

In [53]:
# Export CSV
if len(ldw_hollow) != 0:
    ldw_hollow.to_csv(CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv", index=False)

In [54]:
# Upload to BQ
if len(ldw_hollow) != 0:
    pandas_gbq.to_gbq(
        ldw_hollow,
        f"{DATASET_ID}.lying_deadwood_hollow",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 7928.74it/s]


# Lying Deadwood without hollow

In [55]:
ldw_wo_hollow = extract_ldw_wo_hollow(data)

In [56]:
ldw_wo_hollow.info(), ldw_wo_hollow.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   1826 non-null   object 
 1   repetition  1826 non-null   int64  
 2   type        1826 non-null   object 
 3   class       1826 non-null   object 
 4   diameter    1826 non-null   float64
 5   density     1826 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 85.7+ KB


(None,
   unique_id  repetition type class  diameter  density
 0     308D2           1  tr1    FC      16.5      3.0
 1     308D2           2  tr1    FC      18.3      3.0)

## Export data and upload to BQ

In [57]:
# Export CSV
if len(ldw_wo_hollow) != 0:
    ldw_wo_hollow.to_csv(
        CARBON_POOLS_OUTDIR / "lying_deadwood_wo_hollow.csv", index=False
    )

In [58]:
# Upload to BQ
if len(ldw_wo_hollow) != 0:
    pandas_gbq.to_gbq(
        ldw_wo_hollow,
        f"{DATASET_ID}.lying_deadwood_wo_hollow",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 15887.52it/s]
